In [1]:
import cv2
import numpy as np
from PIL import Image
import copy

In [2]:
cap = cv2.VideoCapture(1)
count = 0
font = cv2.FONT_HERSHEY_SIMPLEX
length = 5
delay = 10
frames = [None] * length


In [3]:
while True:
    ret, frame = cap.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cv2.imshow('frame', frame)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:
while True:
        
    out = cv2.VideoWriter('output.avi', -1, 20.0, (640,480))
    
    ret, frame = cap.read()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_np = np.array(frame)
    backup = np.zeros(frame.shape)
    
    if count < length:
        frames[count] = frame_np
    else:
        backup = frames[count % length]
        frames[count % length] = frame_np
        
    frame_np = frame_np.astype('int32')
    backup = backup.astype('int32')
    diff = np.abs(frame_np - backup)
    diff = diff.astype('uint8')
    frame_np = frame_np.astype('uint8')
    backup = backup.astype('uint8')
    
    
    out.write(diff)
    
    cv2.imshow('diff abs', diff)
    cv2.imshow('frame', frame_np)
    print (white_pixel_counter)
    count += 1
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
out.release()
cv2.destroyAllWindows()

In [9]:
import numpy as np
import cv2
import copy

def findWhitePixels(diff, threshold, height, width):
    side1 = 30
    side2 = 40
    a = np.array([])
    for i in range(0, height - side1, side1):
        for j in range(0, width - side2, side2):
            diff_m = (diff[i:(i + side1), j:(j + side2)]) >= 20
            if np.sum(diff_m) > threshold:
                a = np.append(a, [i, j, i + side1, j + side2])
    return a
            

cap = cv2.VideoCapture(1)
count = 0
font = cv2.FONT_HERSHEY_SIMPLEX
length = 5
samples_number = 5
delay = 30 
samples = np.array([])
calibration = 0
square_positions = [None] * delay
frames = [None] * length
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output15.avi', fourcc, 10.0, (640,480), 0)

while True:
    ret, frame = cap.read()
    height, width, t = frame.shape
    if ret==True:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_np = np.array(frame)
        backup = np.zeros(frame.shape)
            
        if count < length:
            frames[count] = frame_np
        else:
            backup = frames[count % length]
            frames[count % length] = frame_np

        frame_np = frame_np.astype('int32')
        backup = backup.astype('int32')
        diff = np.abs(frame_np - backup)
        diff = diff.astype('uint8')
        frame_np = frame_np.astype('uint8')
        backup = backup.astype('uint8')

        filled_squares = findWhitePixels(diff, 10, 480, 640)
        filled_squares = filled_squares.astype('int32')
        highest = 0
        for i in range(0, filled_squares.shape[0], 4):
            if (filled_squares[i] < filled_squares[highest]) and (filled_squares.shape[0] != 0):
                highest = i
        if filled_squares.shape[0] != 0:
            diff[filled_squares[highest]:filled_squares[highest + 2], filled_squares[highest + 1]:filled_squares[highest + 3]] = 255
        
        
        
        if (count >= length) and ((count - length) < samples_number):
            samples.append(filles_squares[highest])
        if (count - length) == samples_number:
            calibration = np.average(samples)
                
                
            
        
        out.write(diff)
        
            
        cv2.imshow('frame',diff)
        count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()

out.release()

cv2.destroyAllWindows()

In [7]:
ret, frame = cap.read()
h, w, t = frame.shape
print (h, w)
cap.release()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
a = np.array([[1, 2, 3],[4, 5, 6]])
print (a.shape)